In [ ]:
#rnn 구조
import numpy as np
import torch
import torch.nn as nn

class SequenceClassifier(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,n_layers= 4,dropout_p = 0.2):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        
        super().__init__()

        self.nn = nn.LSTM(input_size = input_size,hidden_size =hidden_size,num_layers=n_layers,
                          batch_first = True, dropout = dropout_p, bidirectional = True,)
        #bidirectional = True 는 양방향을 뜻, batch_first = True 는 입력텐서의 모양을 설정 주로 이건 default로 깔고 가는걸 추천
        self.layers = nn.Sequential(
            nn.ReLU(),
            nn.BatchNorm1d(hidden_size*2),
            nn.Linear(hidden_size*2,output_size),
            nn.LogSoftmax(dim=-1),
        )

        def forward(self,x):
            z, _ = self.rnn(x)# 출력과 마지막상태의 은닉을 반환하는데 여기에는 출력만 가져갈거이므로 _로 은닉상태를 반환받는다
            z = z[:,-1] #마지막 순서의 값만 받는다
            y = self.layers(z)#레이어를 통과시켜 마지막 값을 반환받는다
            return y
        
def get_model(input_size,output_size,config, device):
    if config.model == "rnn":
        model = SequenceClassifier(input_size = input_size,hidden_size = config.hiiden_size,output_size = output_size,
                                   n_layers = config.n_layers,dropout = config.dropout_p,)      
    return model                  



In [ ]:
#실제로 쓰는 코딩
import torch
import torch.nn as nn
import torch.nn.functional as F

# 전체 훈련 데이터에 대해 경사 하강법을 2,000회 반복
nb_epochs = 2000
# 모델을 선언 및 초기화. 단순 선형 회귀이므로 input_dim=1, output_dim=1.
model = nn.Linear(1,1)
# optimizer 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])#입력 shape은 (3,1)
y_train = torch.FloatTensor([[2], [4], [6]])

for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward() # backward 연산
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

# 임의의 입력 4를 선언
new_var =  torch.FloatTensor([[4.0]]) #입력텐서의 shape에 맞추어서 입력(1,1)
# 입력한 값 4에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) # forward 연산
# y = 2x 이므로 입력이 4라면 y가 8에 가까운 값이 나와야 제대로 학습이 된 것
print("훈련 후 입력이 4일 때의 예측값 :", pred_y) 



In [ ]:
#선형회귀
import torch.optim as optim
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])
# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

nb_epochs = 1999 # 원하는만큼 경사 하강법을 반복
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x_train * W + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()#0으로 초기화 해주지 않으면 optimizer값이 누적된다
    cost.backward()#backpropagation
    optimizer.step()#w와b 를 업데이트한다
    #requires_grad = True가 적용된 텐서에 연산을 하면, 계산 그래프가 생성되며 backward 함수를 호출하면 그래프로부터 자동으로 미분이 되는 autograd 계산됩니다.

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

In [ ]:
# 다중선형회귀
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  80], 
                               [96,  98,  100],   
                               [73,  66,  70]])  #입력데이터의 형태가 5x3 
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)#0으로 초기화,requires_grad=True를 해야 backpropagation이 가능하다
b = torch.zeros(1, requires_grad=True)#0으로 초기화,requires_grad=True를 해야 backpropagation이 가능하다
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    # 편향 b는 브로드 캐스팅되어 각 샘플에 더해집니다.
    hypothesis = x_train.matmul(W) + b#가설을 행렬곱으로 간단히 정의하였습니다

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))


In [ ]:
#실제로 쓰는 코딩
import torch
import torch.nn as nn
import torch.nn.functional as F

# 전체 훈련 데이터에 대해 경사 하강법을 2,000회 반복
nb_epochs = 2000
# 모델을 선언 및 초기화. 단순 선형 회귀이므로 input_dim=3, output_dim=1.
model = nn.Linear(3,1)
# optimizer 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 0.01, model.parameters()를 사용하여 3개의 w와 b를 전달합니다.
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

# 데이터
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])


for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward() # backward 연산
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

# 임의의 입력 4를 선언
new_var =  torch.FloatTensor([[4.0]]) #입력텐서의 shape에 맞추어서 입력(1,1)
# 입력한 값 4에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) # forward 연산
# y = 2x 이므로 입력이 4라면 y가 8에 가까운 값이 나와야 제대로 학습이 된 것
print("훈련 후 입력이 4일 때의 예측값 :", pred_y) 


#class로 정의 하는법
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):#생성자
        super().__init__()
        self.linear = nn.Linear(3, 1) # 다중 선형 회귀이므로 input_dim=3, output_dim=1.

    def forward(self, x):#실제 동작
        return self.linear(x)


In [ ]:
#전체 데이터에 대해서 한 번에 경사 하강법을 수행하는 방법을 '배치 경사 하강법'이라고 부릅니다. 
#반면, 미니 배치 단위로 경사 하강법을 수행하는 방법을 '미니 배치 경사 하강법'이라고 부릅니다.
#배치 경사 하강법은 경사 하강법을 할 때, 전체 데이터를 사용하므로 가중치 값이 최적값에 수렴하는 과정이 매우 안정적이지만, 
#계산량이 너무 많이 듭니다. 미니 배치 경사 하강법은 경사 하강법을 할 때, 전체 데이터의 일부만을 보고 수행하므로 
#최적값으로 수렴하는 과정에서 값이 조금 헤매기도 하지만 훈련 속도가 빠릅니다.

from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])
dataset = TensorDataset(x_train, y_train)
#데이터로더는 기본적으로 2개의 인자를 입력받는다. 하나는 데이터셋, 미니 배치의 크기입니다. 
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
model = nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 
nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    # print(batch_idx)
    # print(samples)
    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))



In [4]:
#torch.utils.data.Dataset과 torch.utils.data.DataLoader를 제공합니다. 
#이를 사용하면 미니 배치 학습, 데이터 셔플(shuffle), 병렬 처리까지 간단히 수행할 수 있습니다. 
#기본적인 사용 방법은 Dataset을 정의하고, 이를 DataLoader에 전달하는 것입니다.
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
# Dataset 상속
class CustomDataset(Dataset): 
  def __init__(self):
    self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
model = torch.nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):#enumerate 함수를 사용하여 각 배치의 인덱스와 데이터를 가져옵니다.
    # print(batch_idx)
    # print(samples)
    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))



Epoch    0/20 Batch 1/3 Cost: 17388.800781
Epoch    0/20 Batch 2/3 Cost: 12373.408203
Epoch    0/20 Batch 3/3 Cost: 3692.357422
Epoch    1/20 Batch 1/3 Cost: 602.356934
Epoch    1/20 Batch 2/3 Cost: 122.765038
Epoch    1/20 Batch 3/3 Cost: 25.719830
Epoch    2/20 Batch 1/3 Cost: 19.722307
Epoch    2/20 Batch 2/3 Cost: 5.481454
Epoch    2/20 Batch 3/3 Cost: 1.150212
Epoch    3/20 Batch 1/3 Cost: 0.867380
Epoch    3/20 Batch 2/3 Cost: 0.116616
Epoch    3/20 Batch 3/3 Cost: 0.600119
Epoch    4/20 Batch 1/3 Cost: 0.082024
Epoch    4/20 Batch 2/3 Cost: 0.341467
Epoch    4/20 Batch 3/3 Cost: 0.362373
Epoch    5/20 Batch 1/3 Cost: 0.123021
Epoch    5/20 Batch 2/3 Cost: 0.496914
Epoch    5/20 Batch 3/3 Cost: 0.041608
Epoch    6/20 Batch 1/3 Cost: 0.192717
Epoch    6/20 Batch 2/3 Cost: 0.263817
Epoch    6/20 Batch 3/3 Cost: 0.184029
Epoch    7/20 Batch 1/3 Cost: 0.404675
Epoch    7/20 Batch 2/3 Cost: 0.165666
Epoch    7/20 Batch 3/3 Cost: 0.189462
Epoch    8/20 Batch 1/3 Cost: 0.372477
Epoch   

In [ ]:
#로지스틱 선형회귀
#모델 설정하는 방법
model = nn.Sequential(
   nn.Linear(2, 1), # input_dim = 2, output_dim = 1
   nn.Sigmoid() # 출력은 시그모이드 함수를 거친다
)
#클래스로 설정하는 방법
#__init__()에서 모델의 구조와 동적을 정의하는 생성자를 정의합니다. 이는 파이썬에서 객체가 갖는 속성값을 초기화하는 역할로, 
# 객체가 생성될 때 자동으로 호출됩니다. super() 함수를 부르면 여기서 만든 클래스는 nn.Module 클래스의 속성들을 가지고 초기화 됩니다. 
# foward() 함수는 모델이 학습데이터를 입력받아서 forward 연산을 진행시키는 함수입니다. 
# 이 forward() 함수는 model 객체를 데이터와 함께 호출하면 자동으로 실행이됩니다.
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))
model = BinaryClassifier()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

